In [2]:
import torch
import dgl
import numpy as np
import pandas as pd
import os
import pickle
from NLPUtils.preprocessUtils import removePunctuation, removeWord
from NLPUtils.DataModel import Vocabulary
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from tqdm import tqdm

c:\Users\Wen2Tee5\Desktop\Postgraduate\torchPlayGround\torchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ROOT = '.\\R8'
TRAIN = 'train.txt'
TEST = 'test.txt'
MAXLEN = 30
TOKEN = {'<PAD>' : 0, '<UNK>' : 1}

In [4]:
removeWordList = stopwords.words('english')

df_train = pd.read_csv(os.path.join(ROOT, TRAIN), encoding='utf-8',sep='\t', header=None)
df_test = pd.read_csv(os.path.join(ROOT, TRAIN), encoding='utf-8', sep = '\t', header=None)
label2idx = {label : i for i, label in enumerate(df_train[0].unique())}

In [5]:
voc = Vocabulary(TOKENS=TOKEN)

In [6]:
train_sentence = []
test_sentence = []
for sentence in df_train[1]:
    sentence = removePunctuation(sentence=sentence)
    sentence = removeWord(removeWordList=removeWordList, sentence=sentence)
    train_sentence.append(sentence)
for sentence in df_test[1]:
    sentence = removePunctuation(sentence=sentence)
    sentence = removeWord(removeWordList=removeWordList, sentence=sentence)
    test_sentence.append(sentence)

In [7]:
# 計算tf-idf權重並建立詞彙庫
vector = CountVectorizer(token_pattern=r'\S+')
table = vector.fit_transform(train_sentence)
tfidf = TfidfTransformer()
tfidfTable = tfidf.fit_transform(table)


In [8]:
voc = Vocabulary(TOKENS=TOKEN)
voc.addWordList(vector.get_feature_names_out().tolist())

In [9]:
# 將資料encode為idx
train_idxs = []
for sentence in train_sentence:
    data = sentence.split()[:MAXLEN]
    train_idx = [voc.word2idx[word] for word in data]
    while len(train_idx) < MAXLEN:
        train_idx.append(voc.word2idx['<PAD>'])
    train_idxs.append(train_idx)

test_idxs = []
for sentence in test_sentence:
    data = sentence.split()[:MAXLEN]
    test_idx = []
    for word in data:
        if voc.has(word):
            test_idx.append(voc.word2idx[word])
        else:
            test_idx.append(voc.word2idx['<UNK>'])
    while len(test_idx) < MAXLEN:
        test_idx.append(voc.word2idx['<PAD>'])
    test_idxs.append(test_idx)

In [10]:
x_train = np.array(train_idxs, dtype=np.int64)
x_test = np.array(test_idxs, dtype=np.int64)

In [11]:
# PPMI
WINDOW_SIZE = 3
windows_list = []
for sentence in x_train:
    windows = [sentence[i : i + WINDOW_SIZE] for i in range(len(sentence) - WINDOW_SIZE + 1)]
    windows_list += windows
windows_list = np.array(windows_list)

In [15]:
(windows_list == 2).sum()

5